In [22]:
import cv2
import mediapipe as mp
import pandas as pd

In [23]:
# Load the video file
video_path = 'data/test_exercises/test.mp4'
cap = cv2.VideoCapture(video_path)

In [24]:
# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

I0000 00:00:1714080696.681344   25348 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1714080696.682658   27273 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.3-arch1.2), renderer: Mesa Intel(R) UHD Graphics 730 (ADL-S GT1)


In [25]:
# Initialize DataFrame to store keypoints
columns = ['pose_id', 'pose'] + [f'{part}_{coord}' for part in mp_pose.PoseLandmark._member_names_ for coord in ['x', 'y', 'z']]
df = pd.DataFrame(columns=columns)

In [26]:
# Process each frame of the video
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the image to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect poses
    results = pose.process(frame_rgb)

    # Extract keypoints and flatten them
    keypoints = []
    if results.pose_landmarks:
        for landmark in results.pose_landmarks.landmark:
            keypoints.extend([landmark.x, landmark.y, landmark.z])
    else:
        keypoints.extend([0.0] * 75)  # Fill with zeros if no keypoints detected

    # Append pose_id and pose
    keypoints = [frame_count, 'pose'] + keypoints

    # Append keypoints to DataFrame
    df.loc[frame_count] = keypoints

    frame_count += 1

# Save DataFrame to CSV
df.to_csv('output/output_keypoints.csv', index=False)
